# How to Wrap Text on Image using Python
We will be using the extension library `pillow` to draw text on an image. We shall use the following classes from pillow:
- Image : to create an image object for our text 
- ImageDraw: to create a drawing context
- ImageFont: font of the text we will be drawing on the image

In [73]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from random import randint

In [74]:
#create image object from the input image path
try:
    image = Image.open('./eyong.jpg')
 
    
except IOError as e:
    print(e)

### Resize Image base on a given width (500)

In [75]:
# Resize the image 
width = 500
img_w = image.size[0]
img_h = image.size[1]
wpercent = (width/float(img_w))
hsize = int((float(img_h)*float(wpercent)))
rmg = image.resize((width,hsize), Image.ANTIALIAS)
rmg.size

(500, 666)

### Calculate x limits and generate randome x-axis point

In [76]:
# Set x boundry
# Take 8% to the left for min and 50% to the left for max
x_min = (rmg.size[0] * 8) // 100
x_max = (rmg.size[0] * 50) // 100
print(x_min, x_max)

40 250


In [77]:
# Generate the random positioning
ran_x = randint(x_min, x_max)
print(ran_x)

204


In [78]:
# Create font object with the font file and specify desired size
# Font style is `arial` and font size is 20
font_path = 'font/arialbd.ttf'
font = ImageFont.truetype(font=font_path, size=20)

## Spliting text to multiple line captioning
- If text is shorter than the maximum width, then it can fit in one line. Return without splitting
- Split the text using spaces to get each words
- Create short texts by appending words while the width is smaller than the maximum width.

### Split text and find line height
Since we have the randomly generated x-axis, we can split the text to obtain the right amount of lines

In [79]:
def text_wrap(text, font, max_width):
    lines = []
    
    # If the text width is smaller than the image width, then no need to split
    # just add it to the line list and return
    if font.getsize(text)[0]  <= max_width:
        lines.append(text)
    else:
        #split the line by spaces to get words
        words = text.split(' ')
        i = 0
        # append every word to a line while its width is shorter than the image width
        while i < len(words):
            line = ''
            while i < len(words) and font.getsize(line + words[i])[0] <= max_width:
                line = line + words[i]+ " "
                i += 1
            if not line:
                line = words[i]
                i += 1
            lines.append(line)
    return lines
            
        

In [80]:
def draw_text(text):    
    # open the background file
    img = Image.open('./xander_1.jpg')
    
    # size() returns a tuple of (width, height) 
    image_size = img.size 
 
    # create the ImageFont instance
    font_file_path = 'font/arialbd.ttf'
    font = ImageFont.truetype(font_file_path, size=50, encoding="unic")
 
    # get shorter lines
    lines = text_wrap(text, font, image_size[0])
    print(lines) # ['This could be a single line text ', 'but its too long to fit in one. ']
 

Testing our `text_wrap` function to see how well it splits long text

Base on the font, text length and image width, we could split the text into 4 lines

In [81]:
draw_text("This could be a single line text but its too long to fit in one.")

['This could be a ', 'single line text but ', 'its too long to fit in ', 'one. ']


In [82]:
# Caption

text = "This could be a single line text but its too long to fit in one."
lines = text_wrap(text, font, rmg.size[0]-ran_x)
line_height = font.getsize('hg')[1]

### Calculate the Y limit and generate the y-axis point

In [83]:
y_min = (rmg.size[1] * 4) // 100   # 4% from the top
y_max = (rmg.size[1] * 90) //100   # 90% to the bottom
y_max -= (len(lines)*line_height)  # Adjust base on lines and height
ran_y = randint(y_min, y_max)      # Generate random point

### Apply filter to the image
We can add a filter to make the image smooth before drawing text on it

In [84]:
rmg = rmg.filter(ImageFilter.SMOOTH_MORE)

### Draw text on image
Now that we have the radomly generated x and y-axis positions, we have the number of line of text, we have the line height, we can then draw on image

In [85]:
#Create draw object
draw = ImageDraw.Draw(rmg)

In [86]:
color = 'rgb(255,0,0)'  # Red color
x = ran_x
y = y_max

for line in lines:
    draw.text((x,y), line, fill=color, font=font)
    
    y = y + line_height

author = "- Eyong Kevin"
y += 5                        # Add some line space
x += 20                       # Indent a bit to the right
draw.text((x,y), author, fill=color, font=font)
rmg.show()

In [87]:
# Save captioned image
rmg.save('./eyong_kevin.jpg')